In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

---

In [128]:
path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
sub = pd.read_csv(path+'/sub.csv')

In [146]:
train.head();train.tail()

,日期,A厂,B厂,year,month,date,sum
1030,2020/10/27,281198.0,241711.0,2020,10,27,522909.0
1031,2020/10/28,275598.0,240281.0,2020,10,28,515879.0
1032,2020/10/29,268544.0,237466.0,2020,10,29,506010.0
1033,2020/10/30,285484.0,237611.0,2020,10,30,523095.0
1034,2020/10/31,275130.0,236154.0,2020,10,31,511284.0


In [147]:
tests.head();tests.tail()

,日期,A厂,B厂,year,month,date,sum
146,2021/03/27,202,941,2021,03,27,1143
147,2021/03/28,853,985,2021,03,28,1838
148,2021/03/29,623,901,2021,03,29,1524
149,2021/03/30,893,692,2021,03,30,1585
150,2021/03/31,572,885,2021,03,31,1457


In [137]:
train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])


In [148]:
train['sum'] = train['A厂'] + train['B厂']
tests['sum'] = tests['A厂'] + tests['B厂']
train['frac_a'] = train['A厂']/train['sum']
tests['frac_a'] = tests['A厂']/tests['sum']
train['frac_b'] = train['B厂']/train['sum']
tests['frac_b'] = tests['B厂']/tests['sum']

In [187]:
train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)

In [206]:
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())

set()

In [205]:
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

---
# baseline

In [357]:
bs_train = train.iloc[:,0:5]
bs_tests = tests.iloc[:,2:5]

In [358]:
bs_tar = bs_train.iloc[:, 0:2]
bs_train = bs_train.iloc[:, 2:]

In [359]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(n_splits = 5, test_size = .2, train_size = .8, random_state = 1026529)

In [360]:
alg1 = LinearRegression()
cv_results_1 = model_selection.cross_validate(alg1, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_1['test_score'].mean()

-0.14551133194229993

In [361]:
alg2 = Ridge()
cv_results_2 = model_selection.cross_validate(alg2, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()

-0.1455161156470318

In [364]:
bs_mode=alg2.fit(bs_train, y = bs_tar)

In [373]:
sub.iloc[:,1:3] = pd.DataFrame(bs_mode.predict(bs_tests))/100

In [374]:
sub.to_csv('./data/baseline_sub.csv', index=False)

---

## test 2 , the pseudo data

In [2]:
import pandas as pd
import numpy as np
path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
bssub = pd.read_csv(path+'/baseline_sub.csv')

In [3]:
tests.iloc[:, 1:] = bssub.iloc[:, 1:]
train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])
train['sum'] = train['A厂'] + train['B厂']
tests['sum'] = tests['A厂'] + tests['B厂']
train['frac_a'] = train['A厂']/train['sum']
tests['frac_a'] = tests['A厂']/tests['sum']
train['frac_b'] = train['B厂']/train['sum']
tests['frac_b'] = tests['B厂']/tests['sum']
train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

In [5]:
# train.to_csv("./data/train_modified_yan.csv", index = False)
# tests.to_csv("./data/tests_modified_yan.csv", index = False)


,A厂,B厂,year,month,date,sum,frac_a,frac_b
0,200692.0,189490.0,1,1,2018,390182.0,0.514355,0.485645
1,218365.0,211147.0,1,2,2018,429512.0,0.508403,0.491597
2,224501.0,217767.0,1,3,2018,442268.0,0.507613,0.492387
3,216431.0,212636.0,1,4,2018,429067.0,0.504422,0.495578
4,200447.0,209427.0,1,5,2018,409874.0,0.489045,0.510955
...,...,...,...,...,...,...,...,...
1030,281198.0,241711.0,10,27,2020,522909.0,0.537757,0.462243
1031,275598.0,240281.0,10,28,2020,515879.0,0.534230,0.465770
1032,268544.0,237466.0,10,29,2020,506010.0,0.530709,0.469291
1033,285484.0,237611.0,10,30,2020,523095.0,0.545759,0.454241


In [5]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# R_train = pd.read_csv("./data/R_train_modified_yan.csv")
# R_tests = pd.read_csv("./data/R_tests_modified_yan.csv")

In [18]:
wee = list(list(np.repeat(1, 5))+list(np.repeat(0,2)))
wee = wee*148
train['wee'] = wee[0:1035]
train3 = pd.concat([train, train], axis = 0)
train3.iloc[1035:,0] = train3.iloc[:1035,1] # move B chang
train3.iloc[:1035,1] = 0
train3.iloc[1035:,1] = 1

In [39]:
# tests # sunday start
wee = list([0,1,1,1,1,1,0])
wee = wee*22
tests['wee'] = wee[0:151]

tests3 = pd.concat([tests, tests], axis = 0)
tests3.iloc[151:,0] = tests3.iloc[:151,1] # move B chang
tests3.iloc[:151,1] = 0
tests3.iloc[151:,1] = 1

In [41]:
train3 = train3.drop(['sum', 'frac_a', 'frac_b'], axis = 1)
tests3 = tests3.drop(['sum', 'frac_a', 'frac_b'], axis = 1)

In [46]:
train3.to_csv('./data/raw_train.csv', index = False)
tests3.to_csv('./data/raw_tests.csv', index=False)

In [48]:
import optuna

---

# tensorflow

In [3]:
target_train = train.iloc[:,0:2]
target_test  = tests.iloc[:,0:2]
tensor_train = train.iloc[:,2:]
tensor_test  = tests.iloc[:,2:]

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(tensor_train, target_train, test_size=0.25, random_state=1026529)

In [5]:
# x_train
input_shape =  [7]
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import kerastuner as kt
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [6]:
# def model_builder():
#     model = keras.Sequential()
#     layers.Dense(1186, activation='relu', input_shape=input_shape),

#     # hp_units  = hp.Int('units', min_value=593, max_value=1186, step=32)
#     # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
#     # model.add(keras.layers.BatchNormalization()),
#     model.add(keras.layers.Dense(units=593, activation='relu'))
#     model.add(keras.layers.BatchNormalization()),

#     model.add(keras.layers.Dense(units=593, activation='relu'))
#     model.add(keras.layers.BatchNormalization()),
#     model.add(keras.layers.Dense(2))

#     # hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
#     model.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-1),
#                 loss=keras.losses.MeanAbsolutePercentageError())
# # tf.keras.losses.MeanAbsoluteError
#     return model

In [7]:
model = tf.keras.Sequential([
      layers.Dense(1186, activation='relu', input_shape=input_shape),
      keras.layers.Dense(units=593, activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.Dense(units=593, activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.Dense(1)
  ])
# model.add()
model.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-1),
                loss=keras.losses.MeanAbsolutePercentageError())

In [8]:
model.fit(tensor_train, epochs=120)

In [42]:
# model.predict(tensor_test)

# H2o

In [31]:
pd.concat([train.head(), tests.head()], axis = 1)

,A厂,B厂,year,month,date,sum,frac_a,frac_b,A厂,B厂,year,month,date,sum,frac_a,frac_b
0,200692.0,189490.0,1,2018,1,390182.0,0.514355,0.485645,291977.094734,203717.476400,11,2020,1,495694.571134,0.589026,0.410974
1,218365.0,211147.0,1,2018,2,429512.0,0.508403,0.491597,291979.053666,203718.602221,11,2020,2,495697.655887,0.589026,0.410974
2,224501.0,217767.0,1,2018,3,442268.0,0.507613,0.492387,291981.012598,203719.728042,11,2020,3,495700.740640,0.589027,0.410973
3,216431.0,212636.0,1,2018,4,429067.0,0.504422,0.495578,291982.971531,203720.853862,11,2020,4,495703.825393,0.589027,0.410973
4,200447.0,209427.0,1,2018,5,409874.0,0.489045,0.510955,291984.930463,203721.979683,11,2020,5,495706.910146,0.589027,0.410973


In [12]:
train.drop('B厂', axis=1)

,A厂,year,month,date,sum,frac_a,frac_b
0,200692.0,1,2018,1,390182.0,0.514355,0.485645
1,218365.0,1,2018,2,429512.0,0.508403,0.491597
2,224501.0,1,2018,3,442268.0,0.507613,0.492387
3,216431.0,1,2018,4,429067.0,0.504422,0.495578
4,200447.0,1,2018,5,409874.0,0.489045,0.510955
...,...,...,...,...,...,...,...
1030,281198.0,10,2020,27,522909.0,0.537757,0.462243
1031,275598.0,10,2020,28,515879.0,0.534230,0.465770
1032,268544.0,10,2020,29,506010.0,0.530709,0.469291
1033,285484.0,10,2020,30,523095.0,0.545759,0.454241


In [5]:
train3 = pd.concat([train.drop('B厂', axis=1), train.drop('A厂', axis=1)], axis = 0, ignore_index=True)

In [6]:
train3.iloc[:1035,7] = False
train3.iloc[1035:,0] = train3.iloc[1035:,7]
train3.iloc[1035:,7] = True

In [8]:
train3

,A厂,year,month,date,sum,frac_a,frac_b,B厂
0,200692.0,2018,1,1,390182.0,0.514355,0.485645,False
1,218365.0,2018,1,2,429512.0,0.508403,0.491597,False
2,224501.0,2018,1,3,442268.0,0.507613,0.492387,False
3,216431.0,2018,1,4,429067.0,0.504422,0.495578,False
4,200447.0,2018,1,5,409874.0,0.489045,0.510955,False
...,...,...,...,...,...,...,...,...
2065,241711.0,2020,10,27,522909.0,0.537757,0.462243,True
2066,240281.0,2020,10,28,515879.0,0.534230,0.465770,True
2067,237466.0,2020,10,29,506010.0,0.530709,0.469291,True
2068,237611.0,2020,10,30,523095.0,0.545759,0.454241,True


In [7]:
train3.to_csv('./data/full_train.csv', index = False)

In [2]:
%env

{'ELECTRON_RUN_AS_NODE': '1',
 'USER': 'fyenne',
 '__CFBundleIdentifier': 'com.microsoft.VSCode',
 'COMMAND_MODE': 'unix2003',
 'LOGNAME': 'fyenne',
 'PATH': '/opt/homebrew/bin:/Users/fyenne/anaconda3/bin:/Users/fyenne/anaconda3/condabin:/Users/fyenne/.pyenv/plugins/pyenv-virtualenv/shims:/Users/fyenne/.pyenv/shims:/Users/fyenne/.pyenv/plugins/pyenv-virtualenv/shims:/Library/Frameworks/Python.framework/Versions/3.9/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/fyenne/anaconda3/bin/python:/Users/fyenne/python_online/chromedriver:/Library/TeX/texbin:/usr/local/go/bin:/usr/local/aria2/bin:/usr/local/share/dotnet:/opt/X11/bin:~/.dotnet/tools',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.zbLQxgWwzT/Listeners',
 'SHELL': '/bin/zsh',
 'HOME': '/Users/fyenne',
 '__CF_USER_TEXT_ENCODING': '0x1F5:0x0:0x0',
 'TMPDIR': '/var/folders/3g/xmfmfdws08g7_rv6hwgr8dpw0000gn/T/',
 'XPC_SERVICE_NAME': 'application.com.microsoft.VSCode.12993742.12993748',
 'XPC_FLAGS': '0x0',
 'ORIGINAL_XDG_C

In [19]:
%env PATH=/Users/fyenne/miniforge3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/libexec/java_home -v 14.0

env: PATH=/Users/fyenne/miniforge3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/libexec/java_home -v 14.0


In [6]:
!java -version

openjdk version "16" 2021-03-16
OpenJDK Runtime Environment (build 16+14)
OpenJDK 64-Bit Server VM (build 16+14, mixed mode)


In [21]:
!which python

/Users/fyenne/miniforge3/bin/python


In [5]:
!export JAVA_HOME=`/usr/libexec/java_home -v 14.0.2`

In [4]:
import h2o
h2o.init(nthreads=-1, max_mem_size = "8G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "16" 2021-03-16; OpenJDK Runtime Environment (build 16+14); OpenJDK 64-Bit Server VM (build 16+14, mixed mode)
  Starting server from /Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/3g/xmfmfdws08g7_rv6hwgr8dpw0000gn/T/tmpela4373v
  JVM stdout: /var/folders/3g/xmfmfdws08g7_rv6hwgr8dpw0000gn/T/tmpela4373v/h2o_fyenne_started_from_python.out
  JVM stderr: /var/folders/3g/xmfmfdws08g7_rv6hwgr8dpw0000gn/T/tmpela4373v/h2o_fyenne_started_from_python.err


H2OServerError: Server process terminated because of unsupported Java version: Only Java versions 8-14 are supported, system version is 16


In [7]:
import time

In [54]:
kag = pd.read_csv('/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data/kag.csv')

In [55]:
# kag[:151],kag[151:]

In [56]:
bssub['A厂'] = kag[:151]
bssub['B厂'] = kag[151:]

In [57]:
bssub['B厂'] = np.array(kag[151:]);bssub['B厂']

0      176371.60
1      173606.33
2      169540.97
3      180147.70
4      181904.92
         ...    
146    157364.86
147    153134.97
148    153084.25
149    152379.23
150    158839.22
Name: B厂, Length: 151, dtype: float64

In [58]:
bssub.to_csv('kag_sub.csv', index=False)